## This notebook generates an ics file

For some reason the kind folks that run my kids color guard program think it is best to have a schedule in google sheets rather than in say, I don't know.... Maybe a calendar. 

No problem. This fixes that. 

In [5]:
import pandas as pd
from ics import Calendar, Event
import pytz
import re
from datetime import timedelta, datetime

# grab the sheet
SHEET_ID = '1lfg4tMptsJGh-D3sRNOVvAuVTp7q-Zw59VLxKtWX3VA'
SHEET_NAME = 'Complete%20Schedule'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
df = pd.read_csv(url,  skiprows=2)

# fix column headers
df.columns = ['day', 'date', 'time', 'location', 'activity']

# convert to list of dicts
records = df.to_dict('records')

In [6]:
# use this to turn text months into number months
months = {
    'January': 1, 'February': 2, 'March': 3,
    'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9,
    'October': 10, 'November': 11, 'December': 12
}

In [26]:
# take the calendar items and build an ical object then write it out to the file
cal = Calendar()

# grab when i was last run so i can see in the calendar if its working
last_updated_str = str( datetime.now() )

for rt in records:
    e = Event()
    
    # grab records for easy reference
    date = rt['date']
    month = None
    day = None
    year = None
    location = rt[ 'location' ]
    activity = rt[ 'activity' ]
    time = rt[ 'time' ]
    
    # setup activity to display
    if str(activity) != 'nan':
        activity = '\n\n' + str(activity)
    else:
        activity = ''
    
    # get rid of spaces
    time = "".join(time.split()).lower()
    
    # parse out the date
    match = re.search(r'([JFMASOND][a-z]+) +(\d+), +(202\d)', date)
    if match:
        month = months[ match.group(1) ]
        day = int(match.group(2))
        year = int(match.group(3))
    del match
    
    tbd_subject = ''
    if time == 'tbd':
        start_hour = 8
        start_minute = 0
        end_hour = 16
        end_minute = 0
        tbd_subject = ' Exact time is TBD'
    else:
        match = re.search(r'(\d+):?(\d*)([apm]*)-(\d+):?(\d*)([apm]*)', time)
        if match:
            start_hour = int(match.group(1))
            start_minute = match.group(2) or 0
            start_ampm = match.group(3)
            end_hour = int(match.group(4))
            end_minute = match.group(5) or 0
            end_minute = int( end_minute )
            end_ampm = match.group(6)

            # this deals with all the AM/PM BS
            if start_ampm == 'am':
                start_hour = start_hour
            elif start_ampm == 'pm':
                start_hour = start_hour + 12
            elif end_ampm == 'pm':
                start_hour = start_hour + 12

            if end_ampm == 'pm':
                end_hour = end_hour + 12

    # build the event info
    e.name = 'Elie: Color Guard (' + location + ')' + tbd_subject
    e.begin = datetime(year, month, day, start_hour, start_minute, 0, tzinfo=pytz.timezone('US/Eastern')) - timedelta(minutes=15)

    e.end = datetime(year, month, day, end_hour, end_minute, 0, tzinfo=pytz.timezone('US/Eastern')) 
    e.location = location
    #e.url = rt['url']
    e.description = 'Elie: Color Guard -> ' + date + ' -> ' + time + ' -> ' + ' -> ' + location + activity + '\n\nLast synced at: ' + last_updated_str
    #e.organizer = 'bob@ruddy.net'
    
    # add event to the calendar
    cal.events.add(e)

# write out to an ics file
ics_file = 'color.ics'
with open(ics_file, 'w') as my_file:
    my_file.writelines(cal)

# run this to copy to server when finished
    scp /home/ruddy/src/color_guard/color.ics elm:/data/htdocs/elm/color.ics

In [9]:
str( datetime.now() )

'2023-01-19 12:07:06.586333'